Алгоритм:

- строим решетку (grid) и соответствующее ей разбиение Вороного (vor4)
- Ищем все многогранники центрального региона и его диаметр
- составляем список граней центрального многогранника (list_faces)
- перебираю различные подрешетки для каждой ищу приведенный базис, нахожу длину минимального вектора и минимальное расстояние между векторами и их комбинациями:
- Ищем точку s (середина расстояния между точкой (0, 0, 0, 0) и центром ближайшего многогранника
- Нахожу расстояние от точки s до центрального многоранника
- Умножаю это расстояние на 2 и получаю расстояние между многранниками (центральным и ближайшим к нему многогранником из подрешетки)
- составляем словари, ключами которых являются координаты точек s, а значениями - минимальное хроматическое число (dict_det), максимальное запрещенное расстоянием (dict_dist), список матриц (dict_s), соответствующих данной точке s, хроматическому числу и запрещенному расстоянию
- составляем датафрейм (df), обьединяя все эти словари
- составляем список матриц для каждого подходящего хроматического числа (если запрещенное расстояние больше 1)

- central - список вершин центрального многогранника
- coords4 - список координат центров многогранников Вороного

- vor4.vertices - список координат вершин разбиенния Вороного
- vor4.regions - список индексов координат вершин многогранников разбиения Вороного (vor4.vertices)
- vor4.ridge_vertices - координаты вершин 3-мерных граней разбиенния Вороного через их индексы в vor4.vertices
- vor4.ridge_points - список пар центров многогранников Вороного между которыми есть ребро 2х многогранников через индексы в coords4 (по сути - соседние многогранники)

- edge_central_coords - список списков вершин каждой 3-мерной грани центрального многогранника
- edge_central - список индексов (в vor4.vertices)  координат каждой 3-мерной грани 
- list_faces - список 3х мерных граней центрального многогранника с разбивкой каждой 3х мерной грани на 2-мерные
- list_pairs - список центров многогранников Вороного, соседних с центральным (индексы в coords4)

- list_neigh_points - список координат центров многогранников, граничащих с центральным (по сути - вектора нормали к 3-мерным граням центрального многогранника)
- dict_point_edge - словарь: ключ - координаты соседней точки, значение - 3х мерная грань между соседней точкой и (0,0,0,0)
- index_point_edge - индекс грани в списке граней центрального многогранника (edge_central_coords)
- list_ridge_edge - список граней, в том же порядке, что и соседние точки (list_neigh_points) или что и соответствующие вектора нормали к данным граням
- v_norm - список нормированных векторов нормали (в том же порядке, что и изначальные точки(вектора) нормали)

In [2]:
import numpy as np
from scipy.spatial import Voronoi
from scipy.spatial import distance
from sympy import symbols, Eq, solve
import math
import itertools
import copy
from scipy.spatial import Delaunay
import pandas as pd
import sys
from numpy import linalg as la
from itertools import *
from copy import deepcopy
from collections import defaultdict

/Users/konstantinglushkov/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
grid = np.array([
    [2, 0, 0, 0],
    [1, 1, 0, 0],
    [1, 0, 1, 0],
    [1, 0, 0, 1]
], dtype = float)
grid

array([[2., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 0., 1., 0.],
       [1., 0., 0., 1.]])

In [3]:
points_count = 8

coords4 = []
digits = range(- (points_count // 4), points_count - (points_count // 4))

for var in itertools.product(digits, repeat=4):
    coords4.append((grid.T).dot(var).tolist())
    
#coords4 # координаты центров

In [4]:
# строим диаграмму Вороного

vor4 = Voronoi(coords4)

In [5]:
# вершины диаграммы Вороного

vor4.vertices

array([[11. ,  5. , -2. ,  3. ],
       [-5. , -2. ,  2. , -2. ],
       [-3. , -2. , -2. , -2. ],
       ...,
       [16.5,  1.5,  3.5,  3.5],
       [17.5,  1.5,  3.5,  2.5],
       [17. ,  2. ,  5. ,  3. ]])

In [6]:
# регионы

vor4.regions[1]

[497,
 498,
 499,
 500,
 501,
 502,
 503,
 504,
 505,
 506,
 507,
 508,
 509,
 510,
 511,
 512,
 513,
 514,
 662,
 840,
 1067,
 1189,
 1214,
 1387]

In [7]:
# список конечных областей Вороного (-1 в координатах означает, что область бесконечна)

count = 0
vor4.regions_finite = []
for i in vor4.regions:
    if -1 not in i:
        vor4.regions_finite.append(i)
        count += 1
        
print(count)

1297


In [8]:
# максимальный размер региона (по числу вершин)

len_r_max = 0 # размер максимального региона
count_r = 0 # количество таких регионов

for i in range(len(vor4.regions)):
    len_r = len(vor4.regions[i])
    
    if len_r >= len_r_max:
        len_r_max = len_r
        count_r += 1 # считаем количество таких регионов
        
print('количество вершин =', len_r_max, 'количество =', count_r)

количество вершин = 24 количество = 1297


In [9]:
# находим суммарные расстояния от вершин многогранников до (0, 0, 0, 0) (sum_dist), затем находим минимальное 
# значение. Таким образом находим ближайший центральный многогранник

sum_dist_min = 1000

for i in range(1, len(vor4.regions)):
    len_r = len(vor4.regions[i])

    if -1 not in vor4.regions[i]: # регион должен быть конечным
        l = 0
        
        # для i региона перебираем все его вершины и для каждой ищем расстояние до [0.0, 0.0, 0.0, 0.0]
        for j in range(len_r):
            l += distance.euclidean(vor4.vertices[vor4.regions[i][j]], [0.0, 0.0, 0.0, 0.0])

        if l < sum_dist_min:
            sum_dist_min = l
            v_min = i

        
print ('суммарное расстояние =', sum_dist_min, 'индекс центрального региона =', v_min)

суммарное расстояние = 24.0 индекс центрального региона = 1320


In [10]:
# центральный регион

central = vor4.vertices[vor4.regions[v_min]]
central

array([[ 0.5, -0.5, -0.5, -0.5],
       [ 0.5, -0.5, -0.5,  0.5],
       [ 0.5, -0.5,  0.5,  0.5],
       [ 0. ,  0. ,  0. ,  1. ],
       [ 0.5, -0.5,  0.5, -0.5],
       [ 0. , -1. ,  0. ,  0. ],
       [-0.5, -0.5, -0.5,  0.5],
       [ 0. ,  0. ,  1. ,  0. ],
       [-0.5,  0.5,  0.5,  0.5],
       [-0.5,  0.5, -0.5,  0.5],
       [ 0. ,  0. , -1. ,  0. ],
       [-0.5, -0.5, -0.5, -0.5],
       [-0.5,  0.5, -0.5, -0.5],
       [-1. ,  0. ,  0. ,  0. ],
       [-0.5, -0.5,  0.5, -0.5],
       [-0.5,  0.5,  0.5, -0.5],
       [ 0. ,  0. ,  0. , -1. ],
       [-0.5, -0.5,  0.5,  0.5],
       [ 0.5,  0.5,  0.5,  0.5],
       [ 0.5,  0.5, -0.5,  0.5],
       [ 1. ,  0. ,  0. ,  0. ],
       [ 0.5,  0.5,  0.5, -0.5],
       [ 0.5,  0.5, -0.5, -0.5],
       [ 0. ,  1. ,  0. ,  0. ]])

In [11]:
# нахожу длину ребра центрального политопа

d_edge_min = 100
count = 0
for i in range(0, len(central)-1):
    for j in range(i + 1, len(central)):
        d_edge = distance.euclidean(central[i], central[j])
        if d_edge_min >= d_edge:
            d_edge_min = d_edge
            count += 1
            
d_edge_min, count
                           

(1.0, 96)

In [12]:
# индексы вершин центрального региона (координаты vor4.vertices)

vor4.regions[v_min]

[3194,
 3196,
 3922,
 3932,
 5504,
 6827,
 6828,
 7422,
 7423,
 7437,
 7441,
 7720,
 7721,
 7722,
 7729,
 7730,
 7733,
 7894,
 8159,
 8164,
 8165,
 8166,
 8167,
 8174]

In [13]:
# список 3х мерных граней 4х черных многогрнников через индексы вершин в vor4.vertices

vor4.ridge_vertices[0]

[499, 501, 502, 513, 514, 840]

In [14]:
# список вершин многогранников Вороного в координатах

vor4.vertices[499]

array([ 8.5,  4.5, -1.5, -1.5])

In [15]:
# номера координат центров многогранников Вороного (coords4) между которыми проходит грань, перпендикулярная вектору, 
# соединяющего эти центры 

vor4.ridge_points[1]

array([2953, 3016], dtype=int32)

In [16]:
len(vor4.ridge_points)

36834

In [17]:
coords4[2953], coords4[3016]

([8.0, 4.0, -1.0, -1.0], [8.0, 5.0, -1.0, -2.0])

In [18]:
distance.euclidean(coords4[2953], coords4[3016])

1.4142135623730951

In [19]:
coords4[vor4.ridge_points[0][0]]

[8.0, 4.0, -1.0, -1.0]

In [20]:
# нашли индекс точки(центра) центрального многогранника

coords4.index([0.0, 0.0, 0.0, 0.0])

1170

In [21]:
coords4[1170]

[0.0, 0.0, 0.0, 0.0]

In [22]:
# нашли все центры многогранников у которых еть грань между многогранником и центральным многогранником
# индексы координат в coords4

list_pairs = []

for i in vor4.ridge_points:
    if 1170 in i:
        list_pairs.append(i)
        
list_pairs

[array([1169, 1170], dtype=int32),
 array([1162, 1170], dtype=int32),
 array([1617, 1170], dtype=int32),
 array([1610, 1170], dtype=int32),
 array([ 659, 1170], dtype=int32),
 array([1107, 1170], dtype=int32),
 array([ 666, 1170], dtype=int32),
 array([1114, 1170], dtype=int32),
 array([1106, 1170], dtype=int32),
 array([1234, 1170], dtype=int32),
 array([1171, 1170], dtype=int32),
 array([1178, 1170], dtype=int32),
 array([1170, 1618], dtype=int32),
 array([1170,  667], dtype=int32),
 array([1170, 1674], dtype=int32),
 array([1170, 1673], dtype=int32),
 array([1170, 1681], dtype=int32),
 array([1170, 1163], dtype=int32),
 array([1170, 1177], dtype=int32),
 array([1170,  723], dtype=int32),
 array([1170,  730], dtype=int32),
 array([1170,  722], dtype=int32),
 array([1170, 1226], dtype=int32),
 array([1170, 1233], dtype=int32)]

In [23]:
# составляем список центров многогранников, граничащих с центральным

list_neigh_points = [] # список центров многогранников, граничащих с центральным

for pair in list_pairs:
    if pair[0] == 1170:
        list_neigh_points.append(coords4[pair[1]])
    else:
        list_neigh_points.append(coords4[pair[0]])


list_neigh_points

[[-1.0, 0.0, 0.0, -1.0],
 [-1.0, 0.0, -1.0, 0.0],
 [0.0, -1.0, 0.0, -1.0],
 [0.0, -1.0, -1.0, 0.0],
 [-1.0, 0.0, 0.0, 1.0],
 [0.0, -1.0, 0.0, 1.0],
 [-1.0, 0.0, 1.0, 0.0],
 [0.0, -1.0, 1.0, 0.0],
 [-1.0, -1.0, 0.0, 0.0],
 [1.0, 1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 1.0],
 [1.0, 0.0, 1.0, 0.0],
 [1.0, -1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 1.0],
 [1.0, 0.0, -1.0, 0.0],
 [0.0, 0.0, -1.0, -1.0],
 [1.0, 0.0, 0.0, -1.0],
 [0.0, 0.0, -1.0, 1.0],
 [0.0, 0.0, 1.0, -1.0],
 [0.0, 1.0, 0.0, 1.0],
 [0.0, 1.0, 1.0, 0.0],
 [-1.0, 1.0, 0.0, 0.0],
 [0.0, 1.0, -1.0, 0.0],
 [0.0, 1.0, 0.0, -1.0]]

In [24]:
vor4.ridge_vertices[0]

[499, 501, 502, 513, 514, 840]

In [25]:
# трехмерные ячейки 

edge_central = [] # индексы координат вершин 3х мерных граней 


# перебирам 3х мерные грани 
for i in range(len(vor4.ridge_vertices)):
    count = 0

    
    # перебираем вершины каждой 3хмерной грани
    for j in range(len(vor4.ridge_vertices[i])):
        
        # если координата в списке координат центрального региона, увеличиваем count на 1
        if vor4.ridge_vertices[i][j] in vor4.regions[v_min]:
            count += 1 
            
    # если count = количеству вершин в многограннике, то мы нашли 
    if count == len(vor4.ridge_vertices[i]):
        edge_central.append(vor4.ridge_vertices[i])
        
# перевожу индексы вершин в координаты

edge_central_coords = [] # координаты вершин 3х мерных граней

for i in range(len(edge_central)):
    r = []
    for j in range(len(edge_central[i])):
        r.append(vor4.vertices[edge_central[i][j]])
    edge_central_coords.append(np.array(r))
    
edge_central_coords[0]

array([[-0.5, -0.5, -0.5, -0.5],
       [-0.5,  0.5, -0.5, -0.5],
       [-1. ,  0. ,  0. ,  0. ],
       [-0.5, -0.5,  0.5, -0.5],
       [-0.5,  0.5,  0.5, -0.5],
       [ 0. ,  0. ,  0. , -1. ]])

In [26]:
# ищем для каждого центра ближайшую трехмерную грань
# берем каждую точку из списка соседних центральных точек и ищем расстояние от нее до каждой 3х мерной грани
# суммируем и берем минимальное ссумарное расстояние. это и будет 3х мерная грань между точкой (0, 0, 0, 0)
# точкой из списка
# и вектор, соединяющий центры будет вектором нормали к данной 3х мерной грани


dict_point_edge = {} # словарь: ключ - соседняя точка, значение - 3х мерная грань между соседней точкой и (0,0,0,0)
index_point_edge = 0 # индекс грани в списке граней центрального многогранника (edge_central_coords[)
list_ridge_edge = [] # список граней, в том же порядке, что и соседние точки (list_neigh_points)


for point in range(len(list_neigh_points)):
    min_dist_point_edge = 1000

    for edge in range(len(edge_central_coords)):
        dist_point_edge = 0
        
        for coord in edge_central_coords[edge]:
            dist_point_edge += distance.euclidean(coord, list_neigh_points[point])
            
            
        if min_dist_point_edge > dist_point_edge:
            min_dist_point_edge = dist_point_edge
            index_point_edge = edge
            
    dict_point_edge[tuple(list_neigh_points[point])] = edge_central_coords[index_point_edge]
    list_ridge_edge.append(edge_central_coords[index_point_edge])
    
dict_point_edge

{(-1.0,
  0.0,
  0.0,
  -1.0): array([[-0.5, -0.5, -0.5, -0.5],
        [-0.5,  0.5, -0.5, -0.5],
        [-1. ,  0. ,  0. ,  0. ],
        [-0.5, -0.5,  0.5, -0.5],
        [-0.5,  0.5,  0.5, -0.5],
        [ 0. ,  0. ,  0. , -1. ]]),
 (-1.0,
  0.0,
  -1.0,
  0.0): array([[-0.5, -0.5, -0.5,  0.5],
        [-0.5,  0.5, -0.5,  0.5],
        [ 0. ,  0. , -1. ,  0. ],
        [-0.5, -0.5, -0.5, -0.5],
        [-0.5,  0.5, -0.5, -0.5],
        [-1. ,  0. ,  0. ,  0. ]]),
 (0.0,
  -1.0,
  0.0,
  -1.0): array([[ 0.5, -0.5, -0.5, -0.5],
        [ 0.5, -0.5,  0.5, -0.5],
        [ 0. , -1. ,  0. ,  0. ],
        [-0.5, -0.5, -0.5, -0.5],
        [-0.5, -0.5,  0.5, -0.5],
        [ 0. ,  0. ,  0. , -1. ]]),
 (0.0,
  -1.0,
  -1.0,
  0.0): array([[ 0.5, -0.5, -0.5, -0.5],
        [ 0.5, -0.5, -0.5,  0.5],
        [ 0. , -1. ,  0. ,  0. ],
        [-0.5, -0.5, -0.5,  0.5],
        [ 0. ,  0. , -1. ,  0. ],
        [-0.5, -0.5, -0.5, -0.5]]),
 (-1.0,
  0.0,
  0.0,
  1.0): array([[ 0. ,  0. ,  0. , 

In [27]:
list_neigh_points[10], list_ridge_edge[10]

([1.0, 0.0, 0.0, 1.0],
 array([[ 0.5, -0.5, -0.5,  0.5],
        [ 0.5, -0.5,  0.5,  0.5],
        [ 0. ,  0. ,  0. ,  1. ],
        [ 0.5,  0.5,  0.5,  0.5],
        [ 0.5,  0.5, -0.5,  0.5],
        [ 1. ,  0. ,  0. ,  0. ]]))

In [28]:
# проверка на ортогональность

for i in range(5):
    for j in range (1, 6):
        print(np.array(list_neigh_points[10]).dot(np.array(list_ridge_edge[10][i] - list_ridge_edge[10][j])))
         

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [29]:
# список координат каждой 3х мерной грани (координаты даны индексами)

edge_central

[[7720, 7721, 7722, 7729, 7730, 7733],
 [6828, 7437, 7441, 7720, 7721, 7722],
 [3194, 5504, 6827, 7720, 7729, 7733],
 [3194, 3196, 6827, 6828, 7441, 7720],
 [3932, 6828, 7423, 7437, 7722, 7894],
 [3196, 3922, 3932, 6827, 6828, 7894],
 [7422, 7423, 7722, 7729, 7730, 7894],
 [3922, 5504, 6827, 7422, 7729, 7894],
 [6827, 6828, 7720, 7722, 7729, 7894],
 [8159, 8164, 8165, 8166, 8167, 8174],
 [3196, 3922, 3932, 8159, 8164, 8165],
 [3922, 5504, 7422, 8159, 8165, 8166],
 [3194, 3196, 3922, 5504, 6827, 8165],
 [3922, 3932, 7422, 7423, 7894, 8159],
 [3194, 3196, 7441, 8164, 8165, 8167],
 [3194, 7441, 7720, 7721, 7733, 8167],
 [3194, 5504, 7733, 8165, 8166, 8167],
 [3196, 3932, 6828, 7437, 7441, 8164],
 [5504, 7422, 7729, 7730, 7733, 8166],
 [3932, 7423, 7437, 8159, 8164, 8174],
 [7422, 7423, 7730, 8159, 8166, 8174],
 [7423, 7437, 7721, 7722, 7730, 8174],
 [7437, 7441, 7721, 8164, 8167, 8174],
 [7721, 7730, 7733, 8166, 8167, 8174]]

In [30]:
#список координат каждой 3х мерной грани

len(edge_central_coords)

24

In [31]:
# составляем список граней многогранника

list_faces = []

# берем каждый 3х мерный многогранник
for m in edge_central_coords:
    
    # составляем список ребер (расстояние между вершинами, соединенными ребрами = 1)
    list_edges = []
    for i in range(len(m)-1):
        for j in range(i+1, len(m)):
            k = []
            d = distance.euclidean(m[i], m[j])
            if d == 1:
                k.append(i)
                k.append(j)
                list_edges.append(k)
    # перебираем все комбинации ребер и ищем грани (если количество задействованных вершин в комбинации из 3х
    #  ребер = 3, то это грань)                       
    digits = list_edges
    list_faces_one = []  
    for var in itertools.combinations(digits, r=3):
        set_vet = set(var[0] + var[1] + var[2])
        set_vet_coord = []
        if len(set_vet) == 3:
            for i in range(len(set_vet)):
                set_vet_coord.append(m[list(set_vet)[i]])
            list_faces_one.append(set_vet_coord)
               
    list_faces.append(list_faces_one)
    
print('количество граней в одном зх мерном многограннике =', len(list_faces[1]))

количество граней в одном зх мерном многограннике = 8


In [32]:
# список трехмерных граней со списком 2х мерных граней

list_faces[0]

[[array([-0.5, -0.5, -0.5, -0.5]),
  array([-0.5,  0.5, -0.5, -0.5]),
  array([-1.,  0.,  0.,  0.])],
 [array([-0.5, -0.5, -0.5, -0.5]),
  array([-0.5,  0.5, -0.5, -0.5]),
  array([ 0.,  0.,  0., -1.])],
 [array([-0.5, -0.5, -0.5, -0.5]),
  array([-1.,  0.,  0.,  0.]),
  array([-0.5, -0.5,  0.5, -0.5])],
 [array([-0.5, -0.5, -0.5, -0.5]),
  array([-0.5, -0.5,  0.5, -0.5]),
  array([ 0.,  0.,  0., -1.])],
 [array([-0.5,  0.5, -0.5, -0.5]),
  array([-1.,  0.,  0.,  0.]),
  array([-0.5,  0.5,  0.5, -0.5])],
 [array([-0.5,  0.5, -0.5, -0.5]),
  array([-0.5,  0.5,  0.5, -0.5]),
  array([ 0.,  0.,  0., -1.])],
 [array([-1.,  0.,  0.,  0.]),
  array([-0.5, -0.5,  0.5, -0.5]),
  array([-0.5,  0.5,  0.5, -0.5])],
 [array([-0.5, -0.5,  0.5, -0.5]),
  array([-0.5,  0.5,  0.5, -0.5]),
  array([ 0.,  0.,  0., -1.])]]

In [33]:
# диаметр многогранника (ищем максимальное расстояние между вершинами)

max_len = 0

for i in range(len(central) - 1):
    for j in range(i+1, len(central)):
        len_v = distance.euclidean(central[i], central[j])
        if len_v > max_len:
            max_len = len_v
            u1 = i
            u2 = j
               
print('диаметр =', max_len, 'индексы вершин:', u1, ',',  u2) 
# u1, u2 - индексы вершин, расстояние между которыми максимально

диаметр = 2.0 индексы вершин: 0 , 8


In [34]:
# список вершин центрального многогранника
central

array([[ 0.5, -0.5, -0.5, -0.5],
       [ 0.5, -0.5, -0.5,  0.5],
       [ 0.5, -0.5,  0.5,  0.5],
       [ 0. ,  0. ,  0. ,  1. ],
       [ 0.5, -0.5,  0.5, -0.5],
       [ 0. , -1. ,  0. ,  0. ],
       [-0.5, -0.5, -0.5,  0.5],
       [ 0. ,  0. ,  1. ,  0. ],
       [-0.5,  0.5,  0.5,  0.5],
       [-0.5,  0.5, -0.5,  0.5],
       [ 0. ,  0. , -1. ,  0. ],
       [-0.5, -0.5, -0.5, -0.5],
       [-0.5,  0.5, -0.5, -0.5],
       [-1. ,  0. ,  0. ,  0. ],
       [-0.5, -0.5,  0.5, -0.5],
       [-0.5,  0.5,  0.5, -0.5],
       [ 0. ,  0. ,  0. , -1. ],
       [-0.5, -0.5,  0.5,  0.5],
       [ 0.5,  0.5,  0.5,  0.5],
       [ 0.5,  0.5, -0.5,  0.5],
       [ 1. ,  0. ,  0. ,  0. ],
       [ 0.5,  0.5,  0.5, -0.5],
       [ 0.5,  0.5, -0.5, -0.5],
       [ 0. ,  1. ,  0. ,  0. ]])

In [35]:
# нормируем вектора нормали

v_norm = [] # здесь будем хранить нормированные вектора нормали

for vec in list_neigh_points:
    v_norm.append(np.array(vec / np.linalg.norm(vec)))
    
v_norm

[array([-0.70710678,  0.        ,  0.        , -0.70710678]),
 array([-0.70710678,  0.        , -0.70710678,  0.        ]),
 array([ 0.        , -0.70710678,  0.        , -0.70710678]),
 array([ 0.        , -0.70710678, -0.70710678,  0.        ]),
 array([-0.70710678,  0.        ,  0.        ,  0.70710678]),
 array([ 0.        , -0.70710678,  0.        ,  0.70710678]),
 array([-0.70710678,  0.        ,  0.70710678,  0.        ]),
 array([ 0.        , -0.70710678,  0.70710678,  0.        ]),
 array([-0.70710678, -0.70710678,  0.        ,  0.        ]),
 array([0.70710678, 0.70710678, 0.        , 0.        ]),
 array([0.70710678, 0.        , 0.        , 0.70710678]),
 array([0.70710678, 0.        , 0.70710678, 0.        ]),
 array([ 0.70710678, -0.70710678,  0.        ,  0.        ]),
 array([0.        , 0.        , 0.70710678, 0.70710678]),
 array([ 0.70710678,  0.        , -0.70710678,  0.        ]),
 array([ 0.        ,  0.        , -0.70710678, -0.70710678]),
 array([ 0.70710678,  0.

In [36]:
# по грани (list_ridge_edge) центрального многогранника находим нормаль к нему (list_neigh_points), 
# ищем точку, принадлежащую грани (беру сумму координат 2х точек и беру половину) и считаю расстояние

def dist_to_face(index_face, s):
    inner_point = (list_ridge_edge[index_face][3] + list_ridge_edge[index_face][0])/ 2 # точка внутри грани
    #print('inner', inner_point)
    
    dist_face = v_norm[index_face].dot(np.array(inner_point) - np.array(s)) 
    #print('dist', dist_face)
    
    return dist_face

# координаты проекции точки s на гиперплоскость 3х мерной грани

def coords_of_point(normal, dist, s):
    return s - normal * dist
    

def coords_of_point(index, s):
    
    inner_point = (list_ridge_edge[index][3] + list_ridge_edge[index][0])/ 2 # точка внутри грани
    dist_to_face = v_norm[index].dot(np.array(inner_point) - np.array(s)) 
    
    #if dist_to_face > 1:   
    coords = s - v_norm[index] * dist_to_face
    
    
    
    #return dist_face

    return coords

In [37]:
# предподсчитываем М для coords_proj

list_M = []
list_H1 = []
list_H2 = []

for three_d_face in list_faces:
    print('three_d_face', len(three_d_face))
    pre_list_M = []
    pre_list_H1 = []
    pre_list_H2 = []
    for coords in three_d_face:
        
        pre_pre_list_M = []
        pre_pre_list_M.append(coords[0])

        pre_list_l1 = []
        pre_list_l2 = []
        print('coord', coords)
        print('')
        
        for i in range(4):
            pre_list_l1.append(coords[1][i] - coords[0][i])
            pre_list_l2.append(coords[2][i] - coords[0][i])
            
        pre_pre_list_M.append(pre_list_l1)
        pre_pre_list_M.append(pre_list_l2)
        print('prepreM', pre_pre_list_M, i) 
           
        pre_list_M.append(pre_pre_list_M)      
        pre_list_H1.append((np.array(pre_pre_list_M[1])).dot((np.array(pre_pre_list_M).T)))
        pre_list_H2.append((np.array(pre_pre_list_M[2])).dot((np.array(pre_pre_list_M).T)))

    list_M.append(pre_list_M)
    list_H2.append(pre_list_H2)
    list_H1.append(pre_list_H1)
    print('list_M', list_M)
    
list_M[0], list_H1

three_d_face 8
coord [array([-0.5, -0.5, -0.5, -0.5]), array([-0.5,  0.5, -0.5, -0.5]), array([-1.,  0.,  0.,  0.])]

prepreM [array([-0.5, -0.5, -0.5, -0.5]), [0.0, 1.0, 0.0, 0.0], [-0.5, 0.5, 0.5, 0.5]] 3
coord [array([-0.5, -0.5, -0.5, -0.5]), array([-0.5,  0.5, -0.5, -0.5]), array([ 0.,  0.,  0., -1.])]

prepreM [array([-0.5, -0.5, -0.5, -0.5]), [0.0, 1.0, 0.0, 0.0], [0.5, 0.5, 0.5, -0.5]] 3
coord [array([-0.5, -0.5, -0.5, -0.5]), array([-1.,  0.,  0.,  0.]), array([-0.5, -0.5,  0.5, -0.5])]

prepreM [array([-0.5, -0.5, -0.5, -0.5]), [-0.5, 0.5, 0.5, 0.5], [0.0, 0.0, 1.0, 0.0]] 3
coord [array([-0.5, -0.5, -0.5, -0.5]), array([-0.5, -0.5,  0.5, -0.5]), array([ 0.,  0.,  0., -1.])]

prepreM [array([-0.5, -0.5, -0.5, -0.5]), [0.0, 0.0, 1.0, 0.0], [0.5, 0.5, 0.5, -0.5]] 3
coord [array([-0.5,  0.5, -0.5, -0.5]), array([-1.,  0.,  0.,  0.]), array([-0.5,  0.5,  0.5, -0.5])]

prepreM [array([-0.5,  0.5, -0.5, -0.5]), [-0.5, -0.5, 0.5, 0.5], [0.0, 0.0, 1.0, 0.0]] 3
coord [array([-0.5,  0.5

list_M [[[array([-0.5, -0.5, -0.5, -0.5]), [0.0, 1.0, 0.0, 0.0], [-0.5, 0.5, 0.5, 0.5]], [array([-0.5, -0.5, -0.5, -0.5]), [0.0, 1.0, 0.0, 0.0], [0.5, 0.5, 0.5, -0.5]], [array([-0.5, -0.5, -0.5, -0.5]), [-0.5, 0.5, 0.5, 0.5], [0.0, 0.0, 1.0, 0.0]], [array([-0.5, -0.5, -0.5, -0.5]), [0.0, 0.0, 1.0, 0.0], [0.5, 0.5, 0.5, -0.5]], [array([-0.5,  0.5, -0.5, -0.5]), [-0.5, -0.5, 0.5, 0.5], [0.0, 0.0, 1.0, 0.0]], [array([-0.5,  0.5, -0.5, -0.5]), [0.0, 0.0, 1.0, 0.0], [0.5, -0.5, 0.5, -0.5]], [array([-1.,  0.,  0.,  0.]), [0.5, -0.5, 0.5, -0.5], [0.5, 0.5, 0.5, -0.5]], [array([-0.5, -0.5,  0.5, -0.5]), [0.0, 1.0, 0.0, 0.0], [0.5, 0.5, -0.5, -0.5]]], [[array([-0.5, -0.5, -0.5,  0.5]), [0.0, 1.0, 0.0, 0.0], [0.5, 0.5, -0.5, -0.5]], [array([-0.5, -0.5, -0.5,  0.5]), [0.0, 1.0, 0.0, 0.0], [-0.5, 0.5, 0.5, -0.5]], [array([-0.5, -0.5, -0.5,  0.5]), [0.5, 0.5, -0.5, -0.5], [0.0, 0.0, 0.0, -1.0]], [array([-0.5, -0.5, -0.5,  0.5]), [0.0, 0.0, 0.0, -1.0], [-0.5, 0.5, 0.5, -0.5]], [array([-0.5,  0.5, -0

([[array([-0.5, -0.5, -0.5, -0.5]),
   [0.0, 1.0, 0.0, 0.0],
   [-0.5, 0.5, 0.5, 0.5]],
  [array([-0.5, -0.5, -0.5, -0.5]),
   [0.0, 1.0, 0.0, 0.0],
   [0.5, 0.5, 0.5, -0.5]],
  [array([-0.5, -0.5, -0.5, -0.5]),
   [-0.5, 0.5, 0.5, 0.5],
   [0.0, 0.0, 1.0, 0.0]],
  [array([-0.5, -0.5, -0.5, -0.5]),
   [0.0, 0.0, 1.0, 0.0],
   [0.5, 0.5, 0.5, -0.5]],
  [array([-0.5,  0.5, -0.5, -0.5]),
   [-0.5, -0.5, 0.5, 0.5],
   [0.0, 0.0, 1.0, 0.0]],
  [array([-0.5,  0.5, -0.5, -0.5]),
   [0.0, 0.0, 1.0, 0.0],
   [0.5, -0.5, 0.5, -0.5]],
  [array([-1.,  0.,  0.,  0.]), [0.5, -0.5, 0.5, -0.5], [0.5, 0.5, 0.5, -0.5]],
  [array([-0.5, -0.5,  0.5, -0.5]),
   [0.0, 1.0, 0.0, 0.0],
   [0.5, 0.5, -0.5, -0.5]]],
 [[array([-0.5,  1. ,  0.5]),
   array([-0.5,  1. ,  0.5]),
   array([-0.5,  1. ,  0.5]),
   array([-0.5,  1. ,  0.5]),
   array([-0.5,  1. ,  0.5]),
   array([-0.5,  1. ,  0.5]),
   array([-0.5,  1. ,  0.5]),
   array([-0.5,  1. ,  0.5])],
  [array([-0.5,  1. ,  0.5]),
   array([-0.5,  1. ,  0.5]),

In [38]:
# координаты проекции

# face_index - индекс 3-мерной грани, two_d_face - индекс 2-мерной грани, d - точка, проекцию которой надо найти

def coords_proj_pre(face_index, two_d_face, d):
    
    K1 = np.array(list_M[face_index][two_d_face][1]).dot(d)
    K2 = np.array(list_M[face_index][two_d_face][2]).dot(d)
    #print('K', K1, K2)
    # Определение переменных
    x1, x2 = symbols('x1 x2')
    
    # Определение системы уравнений
    equations = [
        Eq(list_H1[face_index][two_d_face][0] + list_H1[face_index][two_d_face][1]*x1 +\
                                                list_H1[face_index][two_d_face][2]*x2, K1),
        Eq(list_H2[face_index][two_d_face][0] + list_H2[face_index][two_d_face][1]*x1 + \
                                                list_H2[face_index][two_d_face][2]*x2, K2)
    ]

    # Решение системы
    symbolic_solution = solve(equations, (x1, x2))

    # Вывод решения
    symbolic_solution
    
    coords = [1]
    coords.append(symbolic_solution[x1])
    coords.append(symbolic_solution[x2])
    
    
    return np.array(list_M[face_index][two_d_face]).T.dot(coords)

# расстояние от точки до отрезка

import math

def distance_point_to_edge(p, a, b):
    
    # скалярное умножение 2х векторов
    def dot_product(v, w):
        sum_coord = 0
        for i in range(len(v)):
            sum_coord += v[i]*w[i]
        return sum_coord
    
    # разность 2х векторов
    def vector(v, w):
        list_dif = []
        for i in range(len(v)):
            k = w[i] - v[i]
            list_dif.append(k)
        return list_dif
    
    # длина вектора
    def magnitude(v):
        sum_sqr = 0
        for i in range(len(v)):
            sum_sqr += v[i]*v[i]
        return math.sqrt(sum_sqr)
    
    # умножение вектора на константу
    def scale(v, const):
        list_const = []
        for i in range(len(v)):
            list_const.append(v[i]*const)
        return list_const

    # сумма 2х векторов
    def add(v, w):
        sum_add = []
        for i in range(len(v)):
            sum_add.append(v[i]+w[i])
        return sum_add

    ab = vector(a, b)
    ap = vector(a, p)

    ab_magnitude = magnitude(ab)
    ap_dot_ab = dot_product(ap, ab)

    if ap_dot_ab <= 0:
        
        # Проекция точки на отрезок находится вне отрезка и ближе к точке a
        return magnitude(ap), False
    
    if ap_dot_ab >= ab_magnitude*ab_magnitude:
        
        # Проекция точки на отрезок находится вне отрезка и ближе к точке b
        dist_b = 0
        for i in range(len(b)):
            dist_b += ((p[i] - b[i])**2)
        return math.sqrt(dist_b), False

    # Проекция точки на отрезок находится внутри отрезка
    projection = add(a, scale(ab, ap_dot_ab/ab_magnitude**2))
    return magnitude(vector(p, projection)), projection

In [39]:
def is_point_on_segment(A, B, P, epsilon=1e-10):
    # Вычисляем разности координат
    delta_x = B[0] - A[0]
    delta_y = B[1] - A[1]
    delta_z = B[2] - A[2]
    delta_w = B[3] - A[3]
    
    # Проверяем, что все координаты пропорциональны
    if abs(delta_x) > epsilon:
        t = (P[0] - A[0]) / delta_x
    elif abs(delta_y) > epsilon:
        t = (P[1] - A[1]) / delta_y
    elif abs(delta_z) > epsilon:
        t = (P[2] - A[2]) / delta_z
    elif abs(delta_w) > epsilon:
        t = (P[3] - A[3]) / delta_w
    else:
        # Если все разности нулевые, то A и B совпадают
        return A == P
    
    # Проверяем, что t находится в диапазоне [0, 1]
    if 0 <= t <= 1:
        # Проверяем, что все координаты соответствуют найденному t
        return (
            abs(A[0] + t * delta_x - P[0]) < epsilon and
            abs(A[1] + t * delta_y - P[1]) < epsilon and
            abs(A[2] + t * delta_z - P[2]) < epsilon and
            abs(A[3] + t * delta_w - P[3]) < epsilon
        )
    else:
        return False


In [40]:
# расстояние от точки до отрезка

import math

def distance_point_to_edge(p, a, b):
    
    # скалярное умножение 2х векторов
    def dot_product(v, w):
        sum_coord = 0
        for i in range(len(v)):
            sum_coord += v[i]*w[i]
        return sum_coord
    
    # разность 2х векторов
    def vector(v, w):
        list_dif = []
        for i in range(len(v)):
            k = w[i] - v[i]
            list_dif.append(k)
        return list_dif
    
    # длина вектора
    def magnitude(v):
        sum_sqr = 0
        for i in range(len(v)):
            sum_sqr += v[i]*v[i]
        return math.sqrt(sum_sqr)
    
    # умножение вектора на константу
    def scale(v, const):
        list_const = []
        for i in range(len(v)):
            list_const.append(v[i]*const)
        return list_const

    # сумма 2х векторов
    def add(v, w):
        sum_add = []
        for i in range(len(v)):
            sum_add.append(v[i]+w[i])
        return sum_add

    ab = vector(a, b)
    ap = vector(a, p)

    ab_magnitude = magnitude(ab)
    ap_dot_ab = dot_product(ap, ab)
    
    d = ap_dot_ab * (1 / ab_magnitude)
    ab_norm = scale(ab, 1 / ab_magnitude)
    coord_proj = add(a, scale(ab_norm, d))

    if is_point_on_segment(a, b, p, epsilon=1e-10):
        # Проекция точки на отрезок находится внутри отрезка
        projection = add(a, scale(ab, ap_dot_ab/ab_magnitude**2))
        return magnitude(vector(p, projection)), projection
        
    else:
        # Проекция точки на отрезок находится вне отрезка
        return magnitude(ap), False


In [42]:
# строим триангуляцию Делоне, используя ее, будем определять находится ли проекция внутри многогранника

delaunay = Delaunay(central)

In [43]:
delaunay.simplices, len(delaunay.simplices)

(array([[18,  6,  1, 10,  5],
        [18,  6,  1,  3,  5],
        [18,  6,  1,  3, 10],
        [18,  6, 11,  5, 13],
        [18,  6, 11, 10, 13],
        [18,  6, 11, 10,  5],
        [18,  0,  1,  5, 20],
        [18,  0,  1, 10, 20],
        [18,  0,  1, 10,  5],
        [18,  0, 11, 10,  5],
        [18,  0, 11, 16,  5],
        [18,  0, 11, 16, 10],
        [18,  2,  1,  3,  5],
        [18, 14, 11, 16,  5],
        [18, 14, 11,  5, 13],
        [18, 14, 11, 16, 13],
        [18,  2,  1,  5, 20],
        [18, 12, 11, 16, 13],
        [18, 12, 11, 10, 13],
        [18, 12, 11, 16, 10],
        [18, 19,  1, 10, 20],
        [18, 19,  1,  3, 10],
        [18, 17,  8,  3, 13],
        [18, 17,  8,  7, 13],
        [18, 17,  2,  3,  5],
        [18, 17,  2,  7,  5],
        [18, 17, 14,  5, 13],
        [18, 17, 14,  7, 13],
        [18, 17, 14,  7,  5],
        [18, 17,  6,  5, 13],
        [18, 17,  6,  3, 13],
        [18, 17,  6,  3,  5],
        [18,  4, 21, 16, 20],
        [1

In [44]:
central[18]

array([0.5, 0.5, 0.5, 0.5])

In [45]:
# Проверка - лежит ли точка внутри многоугольника

def area(x, y, z):
    a = math.sqrt((y[0] - x[0]) ** 2 + (y[1] - x[1]) ** 2 + (y[2] - x[2]) ** 2 + (y[3] - x[3]) ** 2)
    b = math.sqrt((y[0] - z[0]) ** 2 + (y[1] - z[1]) ** 2 + (y[2] - z[2]) ** 2 + (y[3] - z[3]) ** 2)
    c = math.sqrt((x[0] - z[0]) ** 2 + (x[1] - z[1]) ** 2 + (x[2] - z[2]) ** 2 + (x[3] - z[3]) ** 2)
    p = (a + b + c) / 2
    res = math.sqrt(p * (p - a) * (p - b) * (p - c))
    return res

def isInside(list_vert, s):
    count = 0
    for i in range(0, len(list_vert) - 2):
        for j in range(i + 1, len(list_vert) - 1):
            for k in range(j + 1, len(list_vert)):
                
                a = np.array(list_vert[i])
                b = np.array(list_vert[j])
                c = np.array(list_vert[k])

                # Считаем площадь треугольника ABC
                A = area (a, b, c)

                # Считаем площадь треугольника PBC 
                A1 = area (a, b, s)

                # Считаем площадь треугольника PAC 
                A2 = area (a, c, s)

                # Считаем площадь треугольника PAB 
                A3 = area (b, c, s)

                # Проверяем A = A1 + A2 + A3 
                if(A == A1 + A2 + A3):
                    count += 1

    if count > 0:
         return True
    else: 
         return False

### через перебор суммарных расстояний

In [46]:
# для 3-мерной грани
# 1 - находим координаты проекции s на 3-мерную грань
# 2 - проверяем находятся ли координаты проекции на центральном многограннике, то растояние = тому, что посчитали
# 3 - если нет, то находим расстояния до 2-мерных граней и берем минимальное
#    

def dist(s):
    min_dist_s = float('inf') #минимальное расстояние до точки s 
    dist = float('inf') # итоговое расстояние от s до центрального многогранника
    
    # ищем ближайшую 3х мерную грань к точке s, находя сумму растояний до всех вершин грани
    
    min_sum_dist = float('inf') # минимальное суммарное растояние до вершин грани
    
    for index in range(len(list_ridge_edge)): # перебираем все 3-мерные грани
        sum_dist = 0
        
        for index_vert in range(6): # пребираем все вершины каждой 3-мерной грани
            sum_dist += distance.euclidean(s, np.array(list_ridge_edge[index][index_vert], dtype = 'float'))
            
        if sum_dist < min_sum_dist:
            min_sum_dist = sum_dist
            index_face = index
            
         
    # т.о. находим 3-мерную грань (index_face) с минимальным суммарным расстоянием до вершин от точки s
    # находим проекцию s на эту грань
    
    inner_point = (list_ridge_edge[index_face][3] + list_ridge_edge[index_face][0])/ 2 # точка внутри грани
    
    # растояние от s до 3-мерной грани
    dist_to_face = abs(v_norm[index_face].dot(-np.array(inner_point) + np.array(s)))
    
    # координаты проекции
    coords = s - v_norm[index_face] * dist_to_face
    
    #print(coords, '3d')
    
    #print(dist_to_face, coords, '3x')
    
    # проверяем лежит ли точка проекции в центральном многограннике
    simplex = delaunay.find_simplex(coords)

    # вводим отдельно от dist_to_face тк расстояние до гиперплоскости может быть < чем расстояние до central
    #dist = 100 

    if simplex != -1: # если проекция принадлежит центральному многораннику
        if dist_to_face < dist:
            dist = dist_to_face
            coords_to_central = coords
        #print(99)
        
    else: # координаты проекции лежит не на центральном многограннике
        #print(88)
        # тогда ищем расстояние до 2х мерных граней
        # ищем ближайшую 2-мерную грань
        min_sum_dist = float('inf')
        
        for index in range(8):
            sum_dist = 0
            for i in range(3):
                sum_dist += distance.euclidean(s, np.array(list_faces[index_face][index][i], dtype = 'float'))
            if sum_dist < min_sum_dist:
                min_sum_dist = sum_dist
                index_two_d_face = index

        # координаты проекции на 2-мерную грань, зная ближайшую 2- и 3- мерные грани   
        coords_two_d = coords_proj_pre(index_face, index_two_d_face, s) # координаты проекции на 2d грань
        
        #print('index2', index_face, index_two_d_face, coords_two_d, 66)
        
        
        # проверяем, попала ли проекция на грань
        simplex = delaunay.find_simplex(coords_two_d)
        

        if simplex != -1:
            #print(index_face, index_two_d_face, 'simplex')
        #if isInside(list_faces[index_face][index_two_d_face], coords_two_d):
           # print('inside')
            
            dist_2d = distance.euclidean(s, np.array(coords_two_d, dtype = 'float')) # растояние до s
            if dist_2d < dist:
                dist = dist_2d
                coords_to_central = coords_two_d
            
            #print(dist, coords_to_central, 77, '2d')
            
        else:
            #dist = float('inf')
            
            #print(55)
            
            min_sum_dist = float('inf')
            sum_dist = 0
            for i in range(2):
                sum_dist = 0
                for j in range(i + 1, 3):
                    sum_dist = distance.euclidean(s, np.array(list_faces[index_face][index_two_d_face][i], \
                                                               dtype = 'float')) + \
                               distance.euclidean(s, np.array(list_faces[index_face][index_two_d_face][j], \
                                                               dtype = 'float'))
                    if sum_dist < min_sum_dist:
                        min_sum_dist = sum_dist
                        coord_vert_1 = i
                        coord_vert_2 = j
            
            
            
            # находим расстояние до отрезка
            dist_segment = distance_point_to_edge(s, list_faces[index_face][index_two_d_face][coord_vert_1], \
                                                  list_faces[index_face][index_two_d_face][coord_vert_2])
            
            #print(dist_segment, 'отрезок')
            
            # если проекция на отрезок вне его, ищем минимальное трасстояние до вершины
            if dist_segment[1] == False:
                
               # print('не попал в отрезок')
                
                for i in range(3):
                    vert_dist = distance.euclidean(s, np.array(list_faces[index_face][index_two_d_face][i], \
                                                               dtype = 'float'))
                    if vert_dist < dist:
                        dist = vert_dist
                        coords_to_central = list_faces[index_face][index_two_d_face][i]
                    #print('1d', dist, coords_to_central) 
            
            # если проекция на отрезок попала на отрезок
            else:
                #print('dist_segment', dist_segment, index_face, index_two_d_face, coord_vert_1, coord_vert_2)
                if dist > dist_segment[0]:
                    dist = dist_segment[0]
                    coords_to_central = dist_segment[1]

    return dist * 2 / max_len #, coords_to_central

In [47]:
min_d = 2.4

In [48]:
# ищет для подрешетки точку s

def s_point(sub_grid):

    coords4_1 = []
    digits = range(- 2, 2)
    coef = []
    
    # все возможные сочтания из чисел в диапазоне от -2 до 2
    # роверяет, что расстояние между концами векторов при их сочетании не меньше определенного
    for var in itertools.product(digits, repeat=4):
        coords4_1.append(np.dot(sub_grid.T, var).tolist())
        coef.append(var)
    
    min_dist = 100
    coords4_1.remove([0, 0, 0, 0]) #удаляю, чтобы не сравнивать с нулем
    
    # считаю все возмоожные расстояния между точками подрешетки и беру минимальное
    for i in range(len(coords4_1) - 1):
        for j in range(i + 1, len(coords4_1)):
            d_xy = distance.euclidean(coords4_1[i], coords4_1[j])
            
            if d_xy  >= min_d:
                continue

            else:       
                return np.array([0, 0, 0, 0])

        
    # ищем координаты, которые есть в решетке и подрешетке   
    common_coord = []
    min_dist_i = 20
    for i in coords4:
        if i in coords4_1:
            common_coord.append(i)

    # расчитывам расстояние от [0.0, 0.0, 0.0, 0.0] до остальных точек и ищем минимальное расстояние
    min_dist = float('inf')
    for i in range(len(common_coord)):
        dist_centr = distance.euclidean(common_coord[i], [0.0, 0.0, 0.0, 0.0])

        if dist_centr < min_dist:

            min_dist = dist_centr # расстояние от [0.0, 0.0, 0.0, 0.0] до ближайшего другого центра
            min_dist_i = i # индекс в common_coord ближайшего другого центра (от [0.0, 0.0, 0.0, 0.0])

    # находим точку s (середину между центрами многограников)
    if min_dist_i != 20:
        s = np.array(common_coord[min_dist_i], float) * 0.5
        

        return s
        
    return np.array([0, 0, 0, 0])


In [49]:
def gram_schmidt(v):

    w_1 = copy.deepcopy(v[0])
    w_n = copy.deepcopy(w_1)
    w_array = [deepcopy(w_n)]
    
    for n in range(1, len(v)):
        v_n = copy.deepcopy(v[n])
        w_n = copy.deepcopy(v_n)

        for j in range(n):
            w_j = deepcopy(w_array[j])
            if not any(w_j):
                continue
            w_n -= np.dot(v_n, w_j) / np.dot(w_j, w_j) * w_j
        
        w_array += [w_n]
          
    return w_array

In [50]:
d = 0.75

def mu(b_i, b_j):
    return np.dot(b_i, b_j) / np.dot(b_j, b_j) if np.dot(b_j, b_j) != 0 else 0


def LLL(l_basis):

    ortho = gram_schmidt(l_basis)

    k = 1
    n = len(ortho)

    while k < n:
        for j in range(k - 1, -1, -1):
            proj = mu(l_basis[k], ortho[j])
            if abs(proj) > 1/2:
                l_basis[k] -= l_basis[j] * round(proj)
                ortho = copy.deepcopy(gram_schmidt(l_basis))
            #print(l_basis, 111)

        if np.dot(ortho[k], ortho[k]) >= (d - (mu(l_basis[k], ortho[k-1]))**2) * (np.dot(ortho[k-1], ortho[k-1])):
            #print(l_basis, 222)
            k += 1

        else:
            s = copy.deepcopy(l_basis[k-1])
            l_basis[k-1] = copy.deepcopy(l_basis[k])
            l_basis[k] = copy.deepcopy(s)
            ortho = gram_schmidt(l_basis)
            k = max(k-1, 1)
            #print(l_basis, 333)

    return l_basis

In [51]:
# вариант 1 - считаем s - считаем как середина самого короткого вектора lll-базиса

list_mat = []
list_lll = []
list_s1 = []
list_dist = []


# задаю величину определителя
det_mat = 54
mat = np.array([[1, 0, 0, 0], [0, 3, 0, 0], [0, 0, 3, 0], [0, 0, 0, 6]], float)
max_num_col3 = int(mat[3][3])
max_num_col2 = int(mat[2][2])
max_num_col1 = int(mat[1][1])


for i1 in range(max_num_col3):
    mat[2][3] = i1
    print(i1)
    for i2 in range(max_num_col3):
        mat[1][3] = i2
        for i3 in range(max_num_col2):
            mat[1][2] = i3
            for i4 in range(max_num_col3):
                mat[0][3] = i4
                for i5 in range(max_num_col2):
                    mat[0][2] = i5
                    for i6 in range(max_num_col1):
                        mat[0][1] = i6
                        sub_grid = np.dot(mat, grid)
                        basis = LLL(sub_grid)
        
                        # ищем самый короткий вектор
                        min_dist = 100
                        index = 0
                        for i in range(4):
                            d_vec = np.linalg.norm(basis[i])
                            if d_vec < min_dist:
                                min_dist = d_vec
                                index = i
                                
                        # определяем точку s как половину длины самого короткого базисного вектора
                        if min_dist > min_d:
                            s1 = basis[index] * 0.5
                            #print(mat, s1)
                            dist_s1 = dist(s1)
                            if dist_s1 > 1:
                                print(dist_s1, s1, mat)
                                list_dist.append(dist_s1)
                                list_mat.append(mat)
                                list_lll.append(basis)
                                list_s1.append(s1)
 


0
1
2
3
4
5
1.0801234497346435 [-1.  -1.5  0.   0.5] [[1. 2. 2. 0.]
 [0. 3. 0. 5.]
 [0. 0. 3. 5.]
 [0. 0. 0. 6.]]


In [ ]:
# вариант 1 - считаем s - считаем как середина самого короткого вектора lll-базиса

list_mat = []
list_lll = []
list_s1 = []
list_dist = []


# задаю величину определителя
det_mat = 50
mat = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 2, 0], [0, 0, 0, 25]], float)
max_num_col3 = int(mat[3][3])
max_num_col2 = int(mat[2][2])
max_num_col1 = int(mat[1][1])


for i1 in range(max_num_col3):
    mat[2][3] = i1
    print(i1)
    for i2 in range(max_num_col3):
        mat[1][3] = i2
        for i3 in range(max_num_col2):
            mat[1][2] = i3
            for i4 in range(max_num_col3):
                mat[0][3] = i4
                for i5 in range(max_num_col2):
                    mat[0][2] = i5
                    for i6 in range(max_num_col1):
                        mat[0][1] = i6
                        sub_grid = np.dot(mat, grid)
                        basis = LLL(sub_grid)
        
                        # ищем самый короткий вектор
                        min_dist = 100
                        index = 0
                        for i in range(4):
                            d_vec = np.linalg.norm(basis[i])
                            if d_vec < min_dist:
                                min_dist = d_vec
                                index = i
                                
                        # определяем точку s как половину длины самого короткого базисного вектора
                        if min_dist > min_d:
                            s1 = basis[index] * 0.5
                            #print(mat, s1)
                            dist_s1 = dist(s1)
                            if dist_s1 > 1:
                                print(dist_s1, s1, mat)
                                list_dist.append(dist_s1)
                                list_mat.append(mat)
                                list_lll.append(basis)
                                list_s1.append(s1)

In [ ]:
# вариант 1 - считаем s - считаем как середина самого короткого вектора lll-базиса

list_mat = []
list_lll = []
list_s1 = []
list_dist = []


# задаю величину определителя
det_mat = 54
mat = np.array([[1, 0, 0, 0], [0, 2, 0, 0], [0, 0, 3, 0], [0, 0, 0, 9]], float)
max_num_col3 = int(mat[3][3])
max_num_col2 = int(mat[2][2])
max_num_col1 = int(mat[1][1])


for i1 in range(max_num_col3):
    mat[2][3] = i1
    print(i1)
    for i2 in range(max_num_col3):
        mat[1][3] = i2
        for i3 in range(max_num_col2):
            mat[1][2] = i3
            for i4 in range(max_num_col3):
                mat[0][3] = i4
                for i5 in range(max_num_col2):
                    mat[0][2] = i5
                    for i6 in range(max_num_col1):
                        mat[0][1] = i6
                        sub_grid = np.dot(mat, grid)
                        basis = LLL(sub_grid)
        
                        # ищем самый короткий вектор
                        min_dist = 100
                        index = 0
                        for i in range(4):
                            d_vec = np.linalg.norm(basis[i])
                            if d_vec < min_dist:
                                min_dist = d_vec
                                index = i
                                
                        # определяем точку s как половину длины самого короткого базисного вектора
                        if min_dist > min_d:
                            s1 = s_point(basis)
                            #print(mat, s1)
                            dist_s1 = dist(s1)
                            if dist_s1 > 1:
                                print(dist_s1, s1, mat)
                                list_dist.append(dist_s1)
                                list_mat.append(mat)
                                list_lll.append(basis)
                                list_s1.append(s1)
 


In [ ]:
# вариант 1 - считаем s - считаем как середина самого короткого вектора lll-базиса

list_mat = []
list_lll = []
list_s1 = []
list_dist = []


# задаю величину определителя
det_mat = 14
mat = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 2, 0], [0, 0, 0, 7]], float)
max_num_col3 = int(mat[3][3])
max_num_col2 = int(mat[2][2])
max_num_col1 = int(mat[1][1])


for i1 in range(max_num_col3):
    mat[2][3] = i1
    print(i1)
    for i2 in range(max_num_col3):
        mat[1][3] = i2
        for i3 in range(max_num_col2):
            mat[1][2] = i3
            for i4 in range(max_num_col3):
                mat[0][3] = i4
                for i5 in range(max_num_col2):
                    mat[0][2] = i5
                    for i6 in range(max_num_col1):
                        mat[0][1] = i6
                        sub_grid = np.dot(mat, grid)
                        basis = LLL(sub_grid)
        
                        # ищем самый короткий вектор
                        min_dist = 100
                        index = 0
                        for i in range(4):
                            d_vec = np.linalg.norm(basis[i])
                            if d_vec < min_dist:
                                min_dist = d_vec
                                index = i
                                
                        # определяем точку s как половину длины самого короткого базисного вектора
                        if min_dist > min_d:
                            s1 = s_point(basis)
                            #print(mat, s1)
                            dist_s1 = dist(s1)
                            if dist_s1 > 1:
                                print(dist_s1, s1, mat)
                                list_dist.append(dist_s1)
                                list_mat.append(mat)
                                list_lll.append(basis)
                                list_s1.append(s1)
 


In [ ]:
# вариант 2 - считаем s - считаем через функцию s_point

dict_mat_s1_alg = {} # ключ - mat, значение - точка s1 по алгоритму s1
dict_lll_s1 = {} # ключ - приведенный базис, значение - точка s1 как половина от кратчайшего вектора lll-базиса
dict_lll_s1_alg = {} # ключ - приведенный базис, значение - точка s1 по алгоритму s1
dict_mat_lll = {} #  ключ - mat, значение - lll
#dict_dist = defaultdict(list)

list_mat = []
list_lll = []
list_s1 = []
list_s1_alg = []

# задаю величину определителя
det_mat = 54
mat = np.array([[1, 0, 0, 0], [0, 3, 0, 0], [0, 0, 3, 0], [0, 0, 0, 6]], float)
max_num_col3 = int(mat[3][3])
max_num_col2 = int(mat[2][2])
max_num_col1 = int(mat[1][1])


for i1 in range(max_num_col3):
    mat[2][3] = i1
    print(i1)
    for i2 in range(max_num_col3):
        mat[1][3] = i2
        for i3 in range(max_num_col2):
            mat[1][2] = i3
            for i4 in range(max_num_col3):
                mat[0][3] = i4
                for i5 in range(max_num_col2):
                    mat[0][2] = i5
                    for i6 in range(max_num_col1):
                        mat[0][1] = i6
                        sub_grid = np.dot(mat, grid)
                        basis = LLL(sub_grid)
                        print(basis)
                        
                        min_dist = 100
                        index = 0
                        for i in range(4):
                            d_vec = np.linalg.norm(basis[i])
                            if d_vec < min_dist:
                                min_dist = d_vec
                                index = i
                        
                        if min_dist > min_d:
                            s1 = basis[index] * 0.5
                            print(mat, s1)
                            #dict_lll_s1[basis] = s1
                            list_mat.append(mat)
                            list_lll.append(basis)
                            s1_2 = s_point(sub_grid)
                            print(s1_2)
                            list_s1.append(s1)
                            list_s1_alg.append(s1_2)
                            #dict_lll_s1_alg[basis] = s1_2
                            #dict_mat_s1_alg[mat] = s1_2